<a href="https://colab.research.google.com/github/SanthoshToorpu/LLM-s/blob/Unpublished/Bigram_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16
context_length = 32
max_iterations = 5000
evaluation_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
evaluation_iterations = 200
embedding_dimension = 64
num_attention_heads = 4
num_layers = 4
dropout_rate = 0.0

# Set seed
torch.manual_seed(1337)

# Load text data
with open('/content/drive/MyDrive/pride_and_prejudice.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Unique characters in the text
unique_characters = sorted(list(set(text)))
vocab_size = len(unique_characters)

# Character-to-integer mapping
char_to_index = { char: index for index, char in enumerate(unique_characters) }
index_to_char = { index: char for index, char in enumerate(unique_characters) }
encode = lambda s: [char_to_index[c] for c in s]  # Encoder: takes a string, outputs a list of integers
decode = lambda l: ''.join([index_to_char[i] for i in l])  # Decoder: takes a list of integers, outputs a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
split_index = int(0.9 * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

# Data loading
def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    indices = torch.randint(len(data_split) - context_length, (batch_size,))
    x = torch.stack([data_split[i:i + context_length] for i in indices])
    y = torch.stack([data_split[i + 1:i + context_length + 1] for i in indices])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(evaluation_iterations)
        for k in range(evaluation_iterations):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ One head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embedding_dimension, head_size, bias=False)
        self.query = nn.Linear(embedding_dimension, head_size, bias=False)
        self.value = nn.Linear(embedding_dimension, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(embedding_dimension, embedding_dimension)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity """

    def __init__(self, embedding_dimension):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_dimension, 4 * embedding_dimension),
            nn.ReLU(),
            nn.Linear(4 * embedding_dimension, embedding_dimension),
            nn.Dropout(dropout_rate),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, embedding_dimension, num_heads):
        super().__init__()
        head_size = embedding_dimension // num_heads
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(embedding_dimension)
        self.ln1 = nn.LayerNorm(embedding_dimension)
        self.ln2 = nn.LayerNorm(embedding_dimension)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, embedding_dimension)
        self.position_embedding_table = nn.Embedding(context_length, embedding_dimension)
        self.blocks = nn.Sequential(*[Block(embedding_dimension, num_heads=num_attention_heads) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(embedding_dimension)
        self.lm_head = nn.Linear(embedding_dimension, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -context_length:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
model = model.to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iterations):
    if iteration % evaluation_interval == 0 or iteration == max_iterations - 1:
        losses = estimate_loss()
        print(f"step {iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))

0.213083 M parameters
step 0: train loss 4.6120, val loss 4.6059
step 100: train loss 2.6675, val loss 2.6774
step 200: train loss 2.5220, val loss 2.5390
step 300: train loss 2.4026, val loss 2.4330
step 400: train loss 2.3071, val loss 2.3371
step 500: train loss 2.2310, val loss 2.2396
step 600: train loss 2.1495, val loss 2.1633
step 700: train loss 2.0965, val loss 2.1020
step 800: train loss 2.0399, val loss 2.0343
step 900: train loss 1.9792, val loss 1.9850
step 1000: train loss 1.9264, val loss 1.9396
step 1100: train loss 1.8875, val loss 1.8886
step 1200: train loss 1.8673, val loss 1.8565
step 1300: train loss 1.8070, val loss 1.8241
step 1400: train loss 1.7862, val loss 1.7949
step 1500: train loss 1.7699, val loss 1.7723
step 1600: train loss 1.7457, val loss 1.7428
step 1700: train loss 1.7363, val loss 1.7414
step 1800: train loss 1.6957, val loss 1.7098
step 1900: train loss 1.6915, val loss 1.6916
step 2000: train loss 1.6651, val loss 1.6727
step 2100: train loss 1.